In [1]:
import pandas as pd

### Volume Trend Line Chart (Dashboard 1)

In [5]:
import pandas as pd
from datetime import datetime, timedelta

def generate_volume_trend(view="24h"):
    # Load CSVs
    solver_df = pd.read_csv("data/solver_competitions.csv")
    tx_df = pd.read_csv("data/transaction.csv")

    # Merge on auctionId
    merged = pd.merge(solver_df, tx_df, on="auctionId", how="inner")

    # Compute volume
    merged["volume"] = merged[["buyAmountInUSD", "sellAmountInUSD"]].min(axis=1)
    merged["timestamp"] = pd.to_datetime(merged["timestamp"])

    # Filter by time range
    now = merged["timestamp"].max()  # Use latest timestamp in data
    if view == "24h":
        cutoff = now - timedelta(hours=24)
        freq = "1h"
    elif view == "7d":
        cutoff = now - timedelta(days=7)
        freq = "4h"
    else:
        cutoff = merged["timestamp"].min()
        freq = "1H"  # default bin size

    merged = merged[merged["timestamp"] >= cutoff]

    # Bin by time
    merged["time_bin"] = merged["timestamp"].dt.floor(freq)

    # Aggregate volume
    trend = merged.groupby("time_bin")["volume"].sum().reset_index()
    trend["time_bin"] = trend["time_bin"].dt.strftime("%Y-%m-%d %H:%M")
    trend.rename(columns={"time_bin": "timestamp"}, inplace=True)

    # Save to JSON
    output_file = f"out/volume_trend_{view}.json"
    trend.to_json(output_file, orient="records")
    print(f"Wrote {output_file}")

generate_volume_trend("24h")
generate_volume_trend("7d")

Wrote out/volume_trend_24h.json
Wrote out/volume_trend_7d.json
